In [19]:
#Task2.1
import json  # Module for working with JSON data
import copy  # Module for creating deep copies of objects
import random  # Module for generating random numbers
import numpy as np  # Library for numerical computations

# Paths to input files
base_path = "/Users/alaasalem/Downloads/UALFINALS/Alaa/part_2"  # Base directory for input files
duplicates_path = f"{base_path}/Duplicates.txt"  # Path to the Duplicates.txt file
original_path = f"{base_path}/Original.txt"  # Path to the Original.txt file

# Function to load data from JSON files
def load_data():
    """
    Reads data from the specified JSON files and returns it.
    
    Returns:
        tuple: Two objects loaded from the JSON files, representing duplicates and original data.
    """
    with open(duplicates_path, 'r') as f:
        duplicates = json.load(f)  # Load duplicates data from the file
    with open(original_path, 'r') as f:
        original = json.load(f)  # Load original data from the file
    return duplicates, original

# Task 2.1: Function to create an initial population
def create_initial_population(original, population_size=500):
    """
    Generates an initial population by duplicating the original data.

    Args:
        original (object): The original data structure to be duplicated.
        population_size (int, optional): The number of copies to generate. Defaults to 500.

    Returns:
        list: A list containing deep copies of the original data.
    """
    return [copy.deepcopy(original) for _ in range(population_size)]  # Generate a list of deep copies


In [20]:
#Task2.2
def fitness(individual, duplicates):
    """
    Calculates the fitness score of an individual by comparing it to a set of duplicates.
    
    Args:
        individual (list or array): The individual to evaluate, typically represented as a list or array.
        duplicates (dict): A dictionary containing duplicates to compare against, where the values are lists or arrays.

    Returns:
        float: The fitness score, which is the negative of the minimum Manhattan distance 
               between the individual and any duplicate. A higher (less negative) score indicates better fitness.
    """
    # Convert the individual to a NumPy array once, to avoid repeated conversion
    individual_array = np.array(individual)

    # Compute Manhattan distances between the individual and all duplicates
    distances = [
        np.sum(np.abs(individual_array - np.array(duplicate)))
        for duplicate in duplicates.values()
    ]

    # Find the smallest distance (closest match)
    min_distance = min(distances)

    # Return the negative of the minimum distance
    return -min_distance  # Lower distance means better fitness


In [21]:
#Task2.3
def selection(population, fitness_scores):
    """
    Selects the best individual from the population based on fitness scores and returns a new population of clones.
    
    Args:
        population (list): A list containing individuals (e.g., data structures or arrays) in the population.
        fitness_scores (list): A list of fitness scores corresponding to each individual in the population.
        
    Returns:
        list: A new population where each individual is a clone of the best individual.
    """
    # Find the index of the best individual (the one with the highest fitness score)
    best_individual_index = np.argmax(fitness_scores)

    # Get the best individual based on its index
    best_individual = population[best_individual_index]

    # Create a new population consisting of deep copies of the best individual
    new_population = [copy.deepcopy(best_individual) for _ in range(len(population))]

    return new_population


In [22]:
#Task2.4
def mutation(individual, p):
    """
    Applies mutation to an individual with a given probability.
    Each gene in the individual's genome has a chance to be flipped (1 -> 0 or 0 -> 1) based on the mutation probability.
    
    Args:
        individual (list of lists): The individual whose genes will be mutated. It's assumed to be a 2D list or array.
        p (float): The probability of mutating each gene. Must be between 0 and 1.
        
    Returns:
        None: The mutation is applied in-place, modifying the original individual.
    """
    # Iterate over each row in the individual (assumed to be a 2D structure)
    for i in range(len(individual)):
        # Iterate over each element in the row
        for j in range(len(individual[i])):
            # Randomly decide whether to mutate the gene based on probability p
            if random.random() < p:
                # Flip the gene (0 becomes 1, 1 becomes 0)
                individual[i][j] = 1 - individual[i][j]


In [23]:
# Task 2.5: Execute Genetic Algorithm
def run_genetic_algorithm(target, candidates, max_iterations=100, population_size=500, mutation_rate=0.05):
    current_population = create_initial_population(target, population_size)
    is_converged = False

    for gen in range(max_iterations):
        fitness_values = [fitness(ind, candidates) for ind in current_population]
        current_population = selection(current_population, fitness_values)

        for individual in current_population:
            mutation(individual, mutation_rate)

        highest_fitness = max(fitness_values)
        print(f"Generation {gen + 1}: Highest Fitness = {highest_fitness}")

        # Check if all individuals in the population are identical
        if all(np.array_equal(current_population[0], ind) for ind in current_population):
            is_converged = True
            print(f"Convergence reached at generation {gen + 1}.")
            break

    optimal_solution = current_population[0]  # After convergence, all individuals are identical
    return optimal_solution, is_converged

if __name__ == "__main__":
    candidates, target = load_data()
    optimal_candidate, has_converged = run_genetic_algorithm(target, candidates)

    # Find the closest matching duplicate
    closest_match_index = None
    smallest_distance = float('inf')
    for key, candidate in candidates.items():
        distance = np.sum(np.abs(np.array(optimal_candidate) - np.array(candidate)))
        if distance < smallest_distance:
            smallest_distance = distance
            closest_match_index = key

    print(f"The closest matching duplicate is item {closest_match_index} with a distance of {smallest_distance}.")
    print(f"Did the algorithm successfully converge? {'Yes' if has_converged else 'No'}")

    

Generation 1: Highest Fitness = -101
Generation 2: Highest Fitness = -99
Generation 3: Highest Fitness = -99
Generation 4: Highest Fitness = -100
Generation 5: Highest Fitness = -100
Generation 6: Highest Fitness = -98
Generation 7: Highest Fitness = -95
Generation 8: Highest Fitness = -95
Generation 9: Highest Fitness = -90
Generation 10: Highest Fitness = -90
Generation 11: Highest Fitness = -87
Generation 12: Highest Fitness = -82
Generation 13: Highest Fitness = -80
Generation 14: Highest Fitness = -81
Generation 15: Highest Fitness = -82
Generation 16: Highest Fitness = -81
Generation 17: Highest Fitness = -79
Generation 18: Highest Fitness = -80
Generation 19: Highest Fitness = -81
Generation 20: Highest Fitness = -81
Generation 21: Highest Fitness = -82
Generation 22: Highest Fitness = -81
Generation 23: Highest Fitness = -80
Generation 24: Highest Fitness = -78
Generation 25: Highest Fitness = -79
Generation 26: Highest Fitness = -78
Generation 27: Highest Fitness = -77
Generat

In [24]:
# Task 2.6: Reflection on Genetic Algorithm Performance
# The following section reflects on the performance and potential alternatives to the Genetic Algorithm used.

# Reflection on Genetic Algorithm Performance:
# The Genetic Algorithm showed its ability to converge in most scenarios. However, its performance 
# is influenced by factors like mutation rate, population size, and how effectively the fitness function 
# evaluates solutions.

# Alternative methods we could explore include:
# 1. Exhaustive Search:
#    - This method systematically compares the target with all candidates for an exact match.
#    - It's efficient for small datasets but becomes infeasible for larger ones due to its time complexity.

# 2. Heuristic Optimization:
#    - Approaches like simulated annealing offer faster convergence and are better at handling random or noisy datasets.
#    - These methods can avoid the local minima problem, which is often encountered in optimization tasks.

# In summary:
# - Exhaustive search is straightforward and effective for small-scale problems.
# - Simulated annealing provides quick convergence and avoids getting stuck in suboptimal solutions, making it ideal for medium-sized datasets.
# - Genetic algorithms excel at tackling large-scale, complex datasets. Their adaptability and scalability make them powerful tools for solving diverse and intricate problems.

